# General-use Scraper for forums 
# 게시판용 범용 스크레이퍼


## 개요

게시판 페이지의 구조와 타깃 element에 대한 사전 정보 없이 게시글을 쉽게 scraping하기 위해 만들어졌다. 

다만 이 scraper의 초점은 해당 게시판의 과거 게시글까지 모두 저장하는 것이 아니다. 오직 첫 페이지에 올라오는 최근 게시물을 확인하는 기능을 가지고 있다.  

이는 본 scrpaer가 최종적으로 게시판에 새 글이 올라오면 알림을 주는 서비스를 구현하는데 쓰이기 때문이다. 

## 알고리즘

이 scraper는 일반적인 scraping 방식인 특정 사이트에서 특정 정보의 css selector를 입력받아 정보를 얻어오는 방법을 쓰지 않는다. 거꾸로, 사용자가 지정하는 키워드(자연어)를 입력받아 이 키워드가 들어간 element(=게시글)를 찾고, 이 게시글을 토대로 다른 게시글의 css selector을 역산해 데이터를 긁어온다. 

이 때, 사용자가 지정하는 키워드는 사용자가 크롤링하려는 게시판의 게시글 제목에 포함되는 단어어야 한다. 가령 사용자가 취업을 위해 다수의 채용 사이트에서 채용 공고를 모으려 한다고 하자. 대부분의 채용 공고는 게시글 제목에 '~ 채용', '~ 모집', '~ 인턴', 등의 키워드를 포함하고 있을 것이므로 해당 단어들을 키워드로 선택한다. 

본 scraper는 이 키워드들과 크롤링을 하려는 게시판들의 첫 페이지 url을 입력받는다. 그리고 각 게시판에서 키워드가 들어간 element가 있는지 찾는다. 그리고 그 element로부터 parent node를 계속 조사하여 또 다른 키워드가 들어간 element와의 common ancestor를 찾는다. 이를 통해 게시판 페이지의 구조를 사전에 몰라도 역산을 통해 게시글들의 css selector path를 알아낼 수 있다. 

본 scraper는 이와 같은 알고리즘을 쓰기 때문에 다음의 경우 주의해야 한다. 
- 입력한 키워드가 일반적이지 않아 타깃 게시판 첫 페이지에 키워드가 쓰이지 않는 경우
- 입력한 키워드가 게시글 뿐만 아닌 사이트 내 다른 요소에도 들어가 있을 수 있는 경우. (가령 VR게시판에서 VR이라는 키워드로 게시글을 찾으려 할 경우, 게시글이 아닌 메뉴 등에도 키워드가 포함되기에 제대로 작동하지 않을 수 있다.)
- 입력한 키워드가 단일 게시글 element에서 두 번 이상 나타날 수 있는 경우. (이 경우 게시글 단위의 common ancestor를 제대로 찾아낼 수 없다.)
- 입력한 키워드가 들어있는 게시글이 2개 미만일 경우. 

## 기대되는 효과

- 범용적인 크롤링이 가능하기 때문에 여러 사이트로 scale하기에 용이하다. 즉, 각 사이트별로 크롤링을 할 필요가 없다. 
- 크롤링을 잘 모르는 사람도 간단한 인터페이스만 갖추면 쉽게 사용 가능하다. 

In [28]:
# -*- coding: utf-8 -*-

from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

import bs4
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import difflib
from pprint import pprint
import re

### 타깃 사이트를 지정한다. 각 게시판의 첫 페이지 url을 넣는다. 

In [29]:
target_sites = [
    'https://career.kr.kpmg.com/', # 삼정 KPMG
    'https://www2.kiwoom.com/kwa/html/ir/kr/recruit/recruit_info.html', # 키움증권
    'https://kakaopay.recruiter.co.kr/app/jobnotice/list', # 카카오페이 - Chrome dev tool 막아놓음. 
    'https://kakaobank.recruiter.co.kr/app/jobnotice/list', # 카카오뱅크 - Chrome dev tool 막아놓음. 
    'http://www.truefriend.com/main/company/recruit/Recruit.jsp', # 한국투자증권
    'https://fcbfi.org/find-jobs/', # FCBFI 금융학회 금융권 취업 게시판
    'https://www.kofia.or.kr/brd/m_96/list.do', # 금투협 채용안내
]

### 모든 사이트 게시판 첫 페이지에서 2번 이상 언급되는 키워드를 넣는다. 

In [39]:
keywords = [
    '채용',
    '모집',
    '영입',
    '담당',
    
    '인턴',
    '신입',
    '경력',
]

keywords_re = r"|".join(keywords)
regex = re.compile(keywords_re)

regex

re.compile(r'채용|모집|영입|담당|인턴|신입|경력', re.UNICODE)

### 혹 게시판에 http 에러 등이 발생해 접근할 수 없는 것은 아닌지 사전 체크한다. 

In [40]:
def error_checker(site):
    try:
        html = urlopen(site)
    except HTTPError as e:
        print(site, "The server returned an HTTP error, ", e)
        return 1
    except URLError as e:
        print(site, "The server could not be found. ", e)
        return 1
    else: # Success
        return 0

In [41]:
filtered_target_sites = list(map(lambda x: x if error_checker(x)==0 else None, target_sites))
filtered_target_sites = list(filter(None, filtered_target_sites))
target_sites = filtered_target_sites

pprint(target_sites)

['https://career.kr.kpmg.com/',
 'https://www2.kiwoom.com/kwa/html/ir/kr/recruit/recruit_info.html',
 'https://kakaopay.recruiter.co.kr/app/jobnotice/list',
 'https://kakaobank.recruiter.co.kr/app/jobnotice/list',
 'http://www.truefriend.com/main/company/recruit/Recruit.jsp',
 'https://www.kofia.or.kr/brd/m_96/list.do']


### urllib 대신 selenium으로 request를 가져온다. 

In [42]:
driver = webdriver.Chrome()

In [43]:
def delayed_selenium_request(site):
    driver.get(site)
    delay = 3
    site_body = WebDriverWait(driver, timeout=delay).until(EC.visibility_of_all_elements_located((By.TAG_NAME, 'body')))
    req = site_body[0].parent.page_source # .parent returns a Driver object. 
    return req

In [44]:
html_list = []

for i in target_sites:
    html_list.append(delayed_selenium_request(i))

In [45]:
html_list_bs = [bs(x, 'html.parser') for x in html_list]

html2site = {}
for i in range(0, len(html_list_bs)):
    html2site[html_list_bs[i]] = target_sites[i]

len(html_list_bs)

6

### 키워드가 들어간 게시글들을 찾는다. 이 게시물들이 (정확히는 0번째 게시물이) 같은 계층의 다른 게시물들의 css selector path를 추정하는데 쓰인다.

In [46]:
html2target_post = {}

for html_bs in html_list_bs:
    target_posts = html_bs.find_all(text=regex)
    
    if len(target_posts) < 2:
        print('Warning: site[{site}] has less than 2 elements matching the given keywords'.format(site=html2site[html_bs]))
        print("target posts: ", end='')
        print(target_posts) 
        
        continue # Exclude the site that has less than two target posts. 
    
    html2target_post[html_bs] = target_posts

target posts: []


In [47]:
for k, v in html2target_post.items():
    pprint(v)

['채용공고 | 입사지원 | RECRUIT | 회사소개 | 키움증권',
 '채용가이드',
 '채용문의',
 '채용가이드',
 '채용공고',
 '채용문의']
['채용공고 | 채용정보 | 카카오페이',
 '채용정보',
 '채용안내',
 '채용공고',
 '채용문의',
 '채용정보 > 채용공고',
 '채용 공고',
 '대출사업실 사업 기획 담당자 영입',
 '비즈니스 플랫폼 프로덕트 매니저 영입',
 '오프라인 결제 영업 담당자 영입',
 '플랫폼 설계 프로덕트 매니저 영입',
 '법무 행정 업무 지원 어시스턴트 영입',
 '금융 서비스 마케팅 담당자 영입',
 '가맹점 업무지원 담당자 영입',
 '브랜드채널 관리 업무 지원 어시스턴트 영입',
 '카카오페이머니 iOS 개발자 영입']
['영입 프로세스',
 '채용 FAQ',
 '채용 QnA',
 '채용 FAQ',
 '채용 QnA',
 '상시채용',
 '[고객서비스] 대출상담 및 심사 담당자',
 '[인사/경영지원] 인재영입 담당자',
 '[감사] 감사 업무 담당자',
 '[감사] IT감사 업무 담당자',
 '[상품] 카드 회원제도 및 회원 프로세스 기획 담당자',
 '[상품] 전월세대출 상품 기획 및 운영 담당자',
 '[상품] 신용대출 기획 및 운영 담당자',
 '[상품] 전자금융(기업) 기획 담당자',
 '[코어뱅킹] 고객센터 IT 담당자']
['\n\t\t\t\t\t\t채용공지\n\t\t\t\t\t\t',
 '한국투자증권 실무연계 수시인턴 전형',
 '\n\t\t\t\t\t\t채용공지\n\t\t\t\t\t\t',
 '해외투자영업부 경력직 채용 안내',
 '2019 하반기 신입사원 공개채용 최종 면접 합격자 발표',
 '\n\t\t\t\t\t\t채용공지\n\t\t\t\t\t\t',
 '2019 하반기 신입사원 공개채용 1차 면접 합격자 발표',
 '2019 하반기 신입사원 공개채용 직무적합성검사 합격자 발표',
 '2019 하반기 신입사원 공개채용 서류전형 합격자 발표',
 '\n\t\t\t\t\t\t채용공지\n

### 0 번째 게시물을 토대로 키워드가 등장하는 다른 element와의 공통 조상을 찾는다. 

In [48]:
def get_id_class_selector(element):
#     print("Let's check element's attrs", element.attrs)
    if 'id' in element.attrs.keys():
        return element.name + '#' + element.attrs['id']
    elif 'class' in element.attrs.keys() and element.attrs['class'] != []:
        return element.name + '.' + element.attrs['class'][0]
    else:
        return element.name

In [49]:
def find_common_ancestor(child):
    ancestor_path = []
    for parent in child.parents:
        if len(parent.find_all(text=regex)) == 1:
            ancestor_path.append(parent)
            continue
        if len(parent.find_all(text=regex)) > 1:
            return parent, ancestor_path
        else:
            print('Error in find_common_ancestor(): No matching elements to keywords')
            return

In [50]:
html2all_posts = {}
html2css_selector_path = {}

for html_bs in html_list_bs:
    common_ancestor = find_common_ancestor(html2target_post[html_bs][0])
    
    css_selector_path = list(map(get_id_class_selector, common_ancestor[1][::-1]))
    css_selector_path = ">".join(css_selector_path)
    
    all_posts = common_ancestor[0].select(css_selector_path)
    css_selector_path = common_ancestor[0].name + ">" + css_selector_path
    
    html2all_posts[html_bs] = all_posts
    html2css_selector_path[html_bs] = css_selector_path

KeyError: <html lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>KPMG KOREA [Careers]</title>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="0" http-equiv="expires"/>
<meta content="KPMG, SAMJONG" name="keywords"/>
<meta content="KPMG, SAMJONG" name="description"/>
<link href="favicon_new.ico" rel="shortcut icon"/>
<link href="./include/styles.css" media="screen" rel="stylesheet" title="standard" type="text/css"/>
<script language="JavaScript" src="./JScript/Default.js"></script>
<script type="text/javascript">

window.onload = function() {

	getReSize();
}


function usrMngr() {
	var usrMngr_1 = document.getElementById('li_ChngPwd');
	var usrMngr_2 = document.getElementById('li_LogGout');
	if ( usrMngr_1 != null ) {
		usrMngr_1.style.display = '';
		usrMngr_2.style.display = '';
	}
}

function usrMngrUnload() {
	var usrMngr_1 = document.getElementById('li_ChngPwd');
	var usrMngr_2 = documen

### 각 게시판에서 게시글의 css selector가 잘 나오는 것을 볼 수 있다. 

In [236]:
for k, v in html2css_selector_path.items():
    print(v)

tbody>tr>td.td_subject>a
tbody>tr>td.td_subject>a
ul>li.list-item>div.wr-subject>a.item-subject


In [237]:
for k, v in html2all_posts.items():
    pprint(v)

[<a href="http://www.coolenjoy.net/bbs/40/7818">
                    밸브인덱스 한국출시하나요??                    <span class="sound_only">댓글</span><span class="cnt_cmt">[1]</span><span class="sound_only">개</span> </a>,
 <a href="http://www.coolenjoy.net/bbs/40/7816">
                    오큘 리프트s 블랙아웃 현상 때문에 고통받고 있네요                    <span class="sound_only">댓글</span><span class="cnt_cmt">[2]</span><span class="sound_only">개</span> </a>,
 <a href="http://www.coolenjoy.net/bbs/40/7815">
                    vive hdmi 연결했는데 연결 안됐다 뜹니다.                                    </a>,
 <a href="http://www.coolenjoy.net/bbs/40/7802">
                    파이맥스 8K ,영화감상용으론 4K 프로젝터 150인치 보다 나을까요?                    <span class="sound_only">댓글</span><span class="cnt_cmt">[4]</span><span class="sound_only">개</span> </a>,
 <a href="http://www.coolenjoy.net/bbs/40/7801">
                    오딧세이 플러스를 사려고하는데요                    <span class="sound_only">댓글</span><span class="cnt_cmt">[2]</span><span class="sound_only

### css selector을 통해 각 게시판의 첫 게시물 크롤링하기

id, class를 추가했으나, 공지사항과 같이 맨 위에 나타나는 놈을 처리하지 못했다. 

따라서 추후 새로운 글이 추가되었나 확인할 때, 기존 글 DB의 가장 위의 데이터와만 비교하면 안된다. 공지사항처럼 항상 위에 있는 데이터가 있기 때문에 최상단 n개를 긁어와서 기존 DB와 비교하며 뭐가 '새로 추가되었는지' 확인해야한다. 

In [246]:
for k, v in html2css_selector_path.items():
    for i in k.select(v)[:10]:
        a_href = '#'
        if 'href' in i.attrs.keys():
            a_href = i.attrs['href']
        
        print(i.text, " 글 링크: ", a_href)
    print("---"*10)


                    밸브인덱스 한국출시하나요??                    댓글[1]개   글 링크:  http://www.coolenjoy.net/bbs/40/7818

                    오큘 리프트s 블랙아웃 현상 때문에 고통받고 있네요                    댓글[2]개   글 링크:  http://www.coolenjoy.net/bbs/40/7816

                    vive hdmi 연결했는데 연결 안됐다 뜹니다.                                      글 링크:  http://www.coolenjoy.net/bbs/40/7815

                    파이맥스 8K ,영화감상용으론 4K 프로젝터 150인치 보다 나을까요?                    댓글[4]개   글 링크:  http://www.coolenjoy.net/bbs/40/7802

                    오딧세이 플러스를 사려고하는데요                    댓글[2]개   글 링크:  http://www.coolenjoy.net/bbs/40/7801

                    VR 오디세이 플러스 샀습니다.                    댓글[6]개   글 링크:  http://www.coolenjoy.net/bbs/40/7797

                    pico 4k 괜찮나요?                    댓글[2]개   글 링크:  http://www.coolenjoy.net/bbs/40/7795

                    오큘러스 고를 받고 살짝 후회가 듭니다                    댓글[4]개   글 링크:  http://www.coolenjoy.net/bbs/40/7786

                    vive pro 무선 어뎁터 질문요..                    